In [3]:
import requests
import pandas as pd
import json

In [2]:
url = "https://api.eu.navixy.com/v2/tracker/list"

headers = {
    # Added when you pass json=
    # 'Content-Type': 'application/json',
}

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
tracker_list = response.json()
tracker = pd.json_normalize(tracker_list['list'])

In [3]:
tracker.head(2)

,id,label,group_id,tag_bindings,clone,source.status_listing_id,source.id,source.device_id,source.blocked,source.creation_date,source.tariff_id,source.model,source.tariff_end_date,source.phone
0,877766,Steve (MAN),105448,[],False,NaN,545138,979914665171717,False,2021-08-09,1352,atoldrive5,2024-09-01,None
1,877767,John (Scania),105448,[],False,NaN,545139,693299870436055,False,2021-08-09,1352,atoldrive5,2024-09-01,None


In [4]:
tracker.to_csv('data/tracker.csv', index=False)

In [5]:
url = "https://api.eu.navixy.com/v2/employee/list"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
employee_list = response.json()
employee = pd.json_normalize(employee_list['list'])
employee.to_csv('data/employee.csv', index=False)

In [6]:
url = "https://api.eu.navixy.com/v2/vehicle/list"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
vehicle_list = response.json()
vehicle = pd.json_normalize(vehicle_list['list'])
vehicle.to_csv('data/vehicle.csv', index=False)

In [7]:
url = "https://api.eu.navixy.com/v2/tracker/get_states"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
    'trackers': tracker.id.to_list(),
}
response = requests.post(url, headers=headers, json=json_data)
trackers_states = response.json()

In [8]:
df = pd.DataFrame.from_dict(trackers_states['states'], orient='index')
gps = pd.json_normalize(df['gps']).add_prefix('gps.')
gsm = pd.json_normalize(df['gsm']).add_prefix('gsm.')
df = pd.concat([df.reset_index().rename(columns={'index': 'id'}), gps, gsm], axis=1).drop(columns={"gps", "gsm"})

In [9]:
with pd.option_context("display.max_columns", 100, 'display.max_colwidth', 100):
    display(df.head(2))

,id,source_id,connection_status,movement_status,last_update,battery_level,battery_update,inputs,inputs_update,outputs,outputs_update,actual_track_update,additional,gps.updated,gps.signal_level,gps.heading,gps.speed,gps.alt,gps.location.lat,gps.location.lng,gsm.updated,gsm.signal_level,gsm.network_name,gsm.roaming
0,3036047,10230656,offline,parked,2024-07-23 04:11:54,NaN,None,[False],2024-07-23 04:11:54,[],2024-07-23 04:11:54,2024-07-22 23:02:12,NaN,2024-07-23 04:09:12,100,45,0,0,38.133215,-85.768903,None,0,,None
1,3036045,10230654,offline,parked,2024-07-23 04:12:52,NaN,None,"[False, False, False, False, False, False]",2024-07-23 04:12:52,"[False, False, False]",2024-07-23 04:12:52,2024-07-22 21:41:39,"{'event_code': {'value': '0', 'updated': '2024-07-23 04:09:50'}}",2024-07-23 04:09:50,100,45,0,0,30.328325,-97.720808,None,0,,False


In [10]:
df.movement_status.unique()

array(['parked', 'moving', 'stopped'], dtype=object)

In [11]:
df.to_csv('data/trackers_states.csv', index=False)

In [12]:
url = "https://api.eu.navixy.com/v2/tracker/get_fuel"

fuel_states = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    fuel = response.json()
    
    # shouldn't be used, because in some cases "inputs" is an empty list
    # that corrupts other values: 'update_time', 'success'
    # df = pd.json_normalize(fuel, 'inputs', ['update_time', 'success']) 

    df = pd.json_normalize(fuel)
    df = df.explode('inputs')
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    fuel_states.append(df)

df = pd.concat(fuel_states, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('data/fuel_states.csv', index=False)

/tmp/ipykernel_924/659681165.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(fuel_states, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])


In [74]:
url = "https://api.eu.navixy.com/v2/tracker/get_diagnostics"

diagnostics = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    diag = response.json()

    df = pd.json_normalize(diag)
    df = df.explode('inputs')
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    diagnostics.append(df)

df = pd.concat(diagnostics, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('data/diagnostics.csv', index=False)

In [75]:
df

,update_time,success,tracker_id,label,units,name,type,value,units_type,converted_units_type,converted_value
0,None,True,877766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,True,877767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,True,877768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,True,3036043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,True,3036045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,None,True,3036047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,None,True,3036049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-07-23 04:07:21,True,3036056,OBD: Fuel,,obd_fuel,fuel,48.0,percent,None,None
8,2024-07-23 00:39:56,True,3036057,CAN: Fuel (litres),L,can_fuel_litres,fuel,695.0,litre,None,None
9,None,True,3036059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
url = "https://api.eu.navixy.com/v2/tracker/get_inputs"

inputs = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    inp = response.json()

    df = pd.json_normalize(inp)
    df = df.explode('states')
    if pd.isna(df.loc[0, 'states']) == False:
        df = pd.concat([df, df['states'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    inputs.append(df)

df = pd.concat(inputs, axis=0, ignore_index=True).drop(columns=['states', 'user_time'])
df.to_csv('data/inputs.csv', index=False)

In [91]:
df

,update_time,inputs,success,tracker_id,type,name,status,input_number
0,2024-07-23 04:10:01,"[False, False, False]",True,877766,NaN,NaN,NaN,NaN
1,2024-07-23 04:09:59,"[False, False, False]",True,877767,NaN,NaN,NaN,NaN
2,2024-07-23 04:10:01,"[False, False, False]",True,877768,NaN,NaN,NaN,NaN
3,2024-07-23 04:11:54,"[False, False, False]",True,3036043,ignition,Ignition,False,1.0
4,2024-07-23 04:12:52,"[False, False, False, False, False, False]",True,3036045,ignition,Ignition,False,1.0
5,2024-07-23 04:11:54,[False],True,3036047,ignition,Ignition,False,1.0
6,2024-07-23 04:12:52,[],True,3036049,NaN,NaN,NaN,NaN
7,2024-07-23 04:09:26,"[False, False]",True,3036056,ignition,Ignition,False,2.0
8,2024-07-23 04:09:47,"[False, False, False, False, False]",True,3036057,ignition,Ignition,False,1.0
9,2024-07-23 04:11:54,"[False, False, False]",True,3036059,ignition,Ignition,False,2.0


In [97]:
url = "https://api.eu.navixy.com/v2/track/get_last_gps_point"

gps_list = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json())
    df['tracker_id'] = tracker_id
    gps_list.append(df)

df = pd.concat(gps_list, axis=0, ignore_index=True)
df.to_csv('data/last_gps_point.csv', index=False)

In [129]:
url = "https://api.eu.navixy.com/v2/tracker/get_readings"

readings = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    read = response.json()

    df = pd.json_normalize(read)
    df = df.explode('inputs').reset_index(drop=True)
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    readings.append(df)

df = pd.concat(readings, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('data/readings.csv', index=False)

In [14]:
url = "https://api.eu.navixy.com/v2/tracker/stats/mileage/read"
 
json_data = {
    'hash': 'feed000000000000000000000000cafe',
    'trackers': tracker.id.to_list(),
    "from": "2024-06-26 00:00:00", 
    "to": "2024-07-26 00:00:00",
}
response = requests.post(url, headers=headers, json=json_data)
mileage = response.json()

In [15]:
mileage

{'result': {'3036047': {'2024-06-26': {'mileage': 252.43},
   '2024-06-27': {'mileage': 252.63},
   '2024-06-28': {'mileage': 246.38},
   '2024-06-29': {'mileage': 257.19},
   '2024-06-30': {'mileage': 246.0},
   '2024-07-01': {'mileage': 256.98},
   '2024-07-02': {'mileage': 269.11},
   '2024-07-03': {'mileage': 260.5},
   '2024-07-04': {'mileage': 260.71},
   '2024-07-05': {'mileage': 260.72},
   '2024-07-06': {'mileage': 264.85},
   '2024-07-07': {'mileage': 269.94},
   '2024-07-08': {'mileage': 265.6},
   '2024-07-09': {'mileage': 0.0},
   '2024-07-10': None,
   '2024-07-11': None,
   '2024-07-12': {'mileage': 264.64},
   '2024-07-13': {'mileage': 248.17},
   '2024-07-14': {'mileage': 242.76},
   '2024-07-15': {'mileage': 245.23},
   '2024-07-16': {'mileage': 264.37},
   '2024-07-17': {'mileage': 0.0},
   '2024-07-18': {'mileage': 259.52},
   '2024-07-19': {'mileage': 237.19},
   '2024-07-20': {'mileage': 241.39},
   '2024-07-21': {'mileage': 250.0},
   '2024-07-22': {'mileage': 27

In [16]:
dfm = pd.DataFrame.from_dict(mileage['result'], orient='index')
with pd.option_context("display.max_columns", 100, 'display.max_colwidth', 100):
    display(dfm.head(2))

,2024-06-26,2024-06-27,2024-06-28,2024-06-29,2024-06-30,2024-07-01,2024-07-02,2024-07-03,2024-07-04,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,2024-07-15,2024-07-16,2024-07-17,2024-07-18,2024-07-19,2024-07-20,2024-07-21,2024-07-22,2024-07-23,2024-07-24,2024-07-25
3036047,{'mileage': 252.43},{'mileage': 252.63},{'mileage': 246.38},{'mileage': 257.19},{'mileage': 246.0},{'mileage': 256.98},{'mileage': 269.11},{'mileage': 260.5},{'mileage': 260.71},{'mileage': 260.72},{'mileage': 264.85},{'mileage': 269.94},{'mileage': 265.6},{'mileage': 0.0},None,None,{'mileage': 264.64},{'mileage': 248.17},{'mileage': 242.76},{'mileage': 245.23},{'mileage': 264.37},{'mileage': 0.0},{'mileage': 259.52},{'mileage': 237.19},{'mileage': 241.39},{'mileage': 250.0},{'mileage': 275.38},{'mileage': 0.0},None,None
3036045,{'mileage': 300.99},{'mileage': 602.05},{'mileage': 435.6},{'mileage': 467.2},{'mileage': 601.98},{'mileage': 301.03},{'mileage': 902.89},{'mileage': 301.01},{'mileage': 602.0},{'mileage': 451.0},{'mileage': 451.96},{'mileage': 602.02},{'mileage': 343.47},{'mileage': 244.16},None,None,{'mileage': 575.53},{'mileage': 601.89},{'mileage': 568.71},{'mileage': 334.1},{'mileage': 549.5},{'mileage': 301.02},{'mileage': 601.9},{'mileage': 602.05},{'mileage': 526.92},{'mileage': 376.18},{'mileage': 602.07},{'mileage': 0.0},None,None


In [18]:
def get_mileage(x) -> float:
    if x is not None:
        for k,v in x.items():
            return v
    return None
dfm = dfm.map(lambda x: get_mileage(x)).reset_index().rename(columns={'index':'tracker_id'})

In [20]:
dfm.melt(id_vars=["tracker_id"], var_name="date").rename(columns={'value':'mileage'}).to_csv('data/mileage.csv', index=False)

In [ ]:
url = "https://api.eu.navixy.com/v2/tracker/get_diagnostics"

track_lists = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
        "from": "2024-06-26 00:00:00", 
        "to": "2024-07-26 00:00:00",
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json()['list'])
    df['tracker_id'] = tracker_id
    track_lists.append(df)

df = pd.concat(track_lists, axis=0, ignore_index=True)
df.to_csv('data/track.csv', index=False)

In [21]:
url = "https://api.eu.navixy.com/v2/track/list"

track_lists = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
        "from": "2024-06-26 00:00:00", 
        "to": "2024-07-26 00:00:00",
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json()['list'])
    df['tracker_id'] = tracker_id
    track_lists.append(df)

df = pd.concat(track_lists, axis=0, ignore_index=True)
df.to_csv('data/track.csv', index=False)

In [25]:
df.tracker_id.nunique()

14

In [24]:
df.tracker_id.value_counts()

tracker_id
877768     1628
3036043    1241
3036056    1126
877766     1012
3036068     984
3036070     955
3036060     767
3036047     659
3036071     565
3036049     553
877767      450
3036069      86
3036057      60
3036045      39
Name: count, dtype: int64

In [26]:
set(tracker.id.to_list()).difference(set(df.tracker_id.to_list()))

{1228833, 3036059}

In [29]:
dfm = dfm.melt(id_vars=["tracker_id"], var_name="date").rename(columns={'value':'mileage'})

In [34]:
dfm.groupby(['tracker_id'])['mileage'].sum()

tracker_id
1228833       34.81
3036043     6686.52
3036045    12247.23
3036047     5891.69
3036049     6255.88
3036056     7840.96
3036057    24247.30
3036059     3830.03
3036060     5334.08
3036068     8700.18
3036069    21129.16
3036070     4017.95
3036071     5377.40
877766     17502.47
877767     19872.87
877768     18725.67
Name: mileage, dtype: float64

In [123]:
url = "https://api.eu.navixy.com/v2/tracker/get_readings"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
    'tracker_id': tracker.id.to_list()[8],
}
response = requests.post(url, headers=headers, json=json_data)
response.json()

{'user_time': '2024-08-01 19:35:34',
 'inputs': [{'label': 'Board voltage',
   'units': '',
   'name': 'board_voltage',
   'type': 'power',
   'value': 24.9,
   'units_type': 'volt',
   'converted_units_type': None,
   'converted_value': None},
  {'label': 'External temperature',
   'units': '°С',
   'name': 'ext_temp_sensor_1',
   'type': 'temperature',
   'value': 7.9,
   'units_type': 'celsius',
   'converted_units_type': None,
   'converted_value': None}],
 'update_time': '2024-07-23 04:09:47',
 'success': True}

In [128]:
df = pd.json_normalize(response.json())
df = df.explode('inputs')
# if pd.isna(df.loc[0, 'inputs']) == False:
#     df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
pd.isna(df.iloc[0, 1])

False

In [126]:
df

,user_time,inputs,update_time,success
0,2024-08-01 19:35:34,"{'label': 'Board voltage', 'units': '', 'name'...",2024-07-23 04:09:47,True
0,2024-08-01 19:35:34,"{'label': 'External temperature', 'units': '°С...",2024-07-23 04:09:47,True


In [121]:
df

,user_time,inputs,update_time,success
0,2024-08-01 16:18:32,"{'label': 'Board voltage', 'units': '', 'name'...",2024-07-23 04:09:47,True
0,2024-08-01 16:18:32,"{'label': 'External temperature', 'units': '°С...",2024-07-23 04:09:47,True


In [114]:
df

,user_time,inputs,update_time,success
0,2024-08-01 16:14:16,[],None,True


In [116]:
pd.isnull(df.at[0, 'inputs'])

True

In [127]:
response.json()

{'list': [{'id': 22469,
   'start_date': '2024-07-01 00:07:27',
   'avg_speed': 45,
   'end_date': '2024-07-01 00:28:47',
   'length': 16.14,
   'max_speed': 59,
   'norm_fuel_consumed': 2.74,
   'type': 'regular',
   'end_address': '34, Orchard Place, Hawthorne, New Jersey, United States, 07506',
   'start_address': 'Preakness HIlls Country Club, 1050, Ratzer Road, Wayne, New Jersey, United States, 07470',
   'points': 237},
  {'id': 22470,
   'start_date': '2024-07-01 00:37:53',
   'avg_speed': 45,
   'end_date': '2024-07-01 01:04:18',
   'length': 19.7,
   'max_speed': 59,
   'norm_fuel_consumed': 3.35,
   'type': 'regular',
   'end_address': '88, Chadwick Court, Park Ridge, Bergen County, New Jersey, United States, 07656',
   'start_address': '34, Orchard Place, Hawthorne, New Jersey, United States, 07506',
   'points': 279},
  {'id': 22471,
   'start_date': '2024-07-01 01:13:24',
   'avg_speed': 43,
   'end_date': '2024-07-01 01:26:13',
   'length': 9.15,
   'max_speed': 59,
   'n

In [130]:
pd.json_normalize(response.json()['list'])

,id,start_date,avg_speed,end_date,length,max_speed,norm_fuel_consumed,type,end_address,start_address,points
0,22469,2024-07-01 00:07:27,45,2024-07-01 00:28:47,16.14,59,2.74,regular,"34, Orchard Place, Hawthorne, New Jersey, Unit...","Preakness HIlls Country Club, 1050, Ratzer Roa...",237
1,22470,2024-07-01 00:37:53,45,2024-07-01 01:04:18,19.70,59,3.35,regular,"88, Chadwick Court, Park Ridge, Bergen County,...","34, Orchard Place, Hawthorne, New Jersey, Unit...",279
2,22471,2024-07-01 01:13:24,43,2024-07-01 01:26:13,9.15,59,1.56,regular,"740, Pascack Road, Paramus, New Jersey, United...","88, Chadwick Court, Park Ridge, Bergen County,...",138
3,22472,2024-07-01 01:35:20,45,2024-07-01 01:49:24,10.44,59,1.77,regular,"158, Baker Avenue, Bergenfield, New Jersey, Un...","740, Pascack Road, Paramus, New Jersey, United...",157
4,22473,2024-07-01 01:58:30,46,2024-07-01 02:12:35,10.69,59,1.82,regular,"43, Van Orden Road, Harrington Park, Bergen Co...","158, Baker Avenue, Bergenfield, New Jersey, Un...",159
...,...,...,...,...,...,...,...,...,...,...,...
783,23252,2024-07-23 01:30:34,46,2024-07-23 01:52:41,17.03,59,2.90,regular,"Cypress Avenue, New York, United States, 11232","Marine Parkway Bridge, Marine Parkway–Gil Hodg...",239
784,23253,2024-07-23 02:01:50,44,2024-07-23 02:25:17,17.19,59,2.92,regular,NYPD Air Operations Heliport (Floyd Bennett Fi...,"Cypress Avenue, New York, United States, 11232",250
785,23254,2024-07-23 02:34:25,45,2024-07-23 02:58:03,17.92,59,3.05,regular,"Nassau Expressway, New York, United States, 11420",NYPD Air Operations Heliport (Floyd Bennett Fi...,257
786,23255,2024-07-23 03:07:12,46,2024-07-23 03:35:55,22.09,59,3.76,regular,"Red Trail, New York, United States, 11229","Nassau Expressway, New York, United States, 11420",317


In [58]:
df = pd.json_normalize(fuel)
df = df.explode('inputs')
if pd.isna(df.loc[0, 'inputs']) == False:
    df = pd.concat([df.drop(['inputs'], axis=1), df['inputs'].apply(pd.Series)], axis=1)

In [45]:
df = df.astype({"id": int}).merge(pd.json_normalize(response_json['list'])[['id', 'label']], on='id')